# Secuencial

In [ ]:
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

def cargar_archivo_fna(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read()
    return contenido

# Cargar los dos archivos .fna
archivo1 = 'E_coli.fna'
archivo2 = 'Salmonella.fna'

# Medir el tiempo de ejecución
start_time = time.time()

contenido1 = cargar_archivo_fna(archivo1)
contenido2 = cargar_archivo_fna(archivo2)

elapsed_time = time.time() - start_time
print(f'Tiempo de carga: {elapsed_time:.2f} segundos')

# Crear el dotplot
def dotplot(secuencia1, secuencia2):
    matriz_dotplot = [[1 if secuencia1[i] == secuencia2[j] else 0 for j in range(len(secuencia2))] for i in range(len(secuencia1))]
    plt.imshow(matriz_dotplot, cmap='Greys', origin='lower')
    plt.xlabel('Secuencia 2')
    plt.ylabel('Secuencia 1')
    plt.title('Dotplot')
    plt.show()

# Mostrar barra de progreso
total_files = 2
progress_bar = tqdm(total=total_files, desc='Cargando archivos')

dotplot(contenido1, contenido2)
progress_bar.update(1)

progress_bar.close()

In [3]:
%%writefile secuencial.py
import argparse
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import lil_matrix


# Función para generar el dotplot secuencialmente
def sequential_dotplot(seq1, seq2):
    n = len(seq1)
    m = len(seq2)

    dotplot = lil_matrix((n, m), dtype=bool)

    for i in range(n):
        for j in range(m):
            if seq1[i] == seq2[j]:
                dotplot[i, j] = True

    return dotplot

# Función principal para ejecutar el dotplot secuencial
def run_dotplot(seq1_file, seq2_file):
    # Leer las secuencias desde los archivos FASTA
    seq1 = str(next(SeqIO.parse(seq1_file, "fasta")).seq)
    seq2 = str(next(SeqIO.parse(seq2_file, "fasta")).seq)

    dotplot = sequential_dotplot(seq1, seq2)

    # Generar el gráfico del dotplot
    plt.imshow(dotplot, cmap="Greys")
    plt.title("Dotplot")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()

if __name__ == "__main__":
    # Configurar los argumentos de línea de comandos
    parser = argparse.ArgumentParser(description="Dotplot Generator")
    parser.add_argument("seq1_file", type=str, help="Archivo FASTA de la secuencia 1")
    parser.add_argument("seq2_file", type=str, help="Archivo FASTA de la secuencia 2")
    args = parser.parse_args()

    # Ejecutar el dotplot con los argumentos proporcionados
    run_dotplot(args.seq1_file, args.seq2_file)

Overwriting secuencial.py


In [5]:
!python3 secuencial.py E_coli.fna Salmonella.fna

# Multiprocessing

In [ ]:
%%writefile multiprocessing.py
import argparse
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import lil_matrix

# Función para generar el dotplot utilizando una matriz dispersa
def sparse_dotplot(seq1, seq2):
    n = len(seq1)
    m = len(seq2)
    dotplot = lil_matrix((n, m), dtype=bool)

    for i in range(n):
        for j in range(m):
            if seq1[i] == seq2[j]:
                dotplot[i, j] = True

    return dotplot

# Función principal para ejecutar el dotplot
def run_dotplot(seq1_file, seq2_file):
    # Leer las secuencias desde los archivos FASTA
    seq1 = str(next(SeqIO.parse(seq1_file, "fasta")).seq)
    seq2 = str(next(SeqIO.parse(seq2_file, "fasta")).seq)

    dotplot = sparse_dotplot(seq1, seq2)

    # Generar el gráfico del dotplot
    plt.imshow(dotplot.toarray(), cmap="Greys")
    plt.title("Dotplot")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()

if _name_ == "_main_":
    # Configurar los argumentos de línea de comandos
    parser = argparse.ArgumentParser(description="Dotplot Generator")
    parser.add_argument("seq1_file", type=str, help="Archivo FASTA de la secuencia 1")
    parser.add_argument("seq2_file", type=str, help="Archivo FASTA de la secuencia 2")
    args = parser.parse_args()

    # Ejecutar el dotplot con los argumentos proporcionados
    run_dotplot(args.seq1_file, args.seq2_file)

In [ ]:
!python3 multiprocessing.py E_coli.fna Salmonella.fna

# MPI4PY

In [9]:
%%writefile dotplot_mpi.py
import argparse
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
from mpi4py import MPI

# Función para generar el dotplot utilizando mpi4py
def distributed_dotplot(seq1, seq2):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    n = len(seq1)
    m = len(seq2)
    dotplot = np.zeros((n, m))

    # Dividir las filas de trabajo entre los procesos
    chunk_size = n // size
    start_row = rank * chunk_size
    end_row = start_row + chunk_size if rank < size - 1 else n

    for i in range(start_row, end_row):
        for j in range(m):
            if seq1[i] == seq2[j]:
                dotplot[i, j] = 1

    # Recopilar los resultados parciales en el proceso raíz
    dotplot = comm.gather(dotplot, root=0)

    if rank == 0:
        dotplot = np.concatenate(dotplot)

    return dotplot

# Función principal para ejecutar el dotplot
def run_dotplot(seq1_file, seq2_file):
    # Leer las secuencias desde los archivos FASTA
    seq1 = str(next(SeqIO.parse(seq1_file, "fasta")).seq)
    seq2 = str(next(SeqIO.parse(seq2_file, "fasta")).seq)

    dotplot = distributed_dotplot(seq1, seq2)

    # Generar el gráfico del dotplot
    plt.imshow(dotplot, cmap="Greys")
    plt.title("Dotplot")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()

if __name__ == "__main__":
    # Configurar los argumentos de línea de comandos
    parser = argparse.ArgumentParser(description="Dotplot Generator")
    parser.add_argument("seq1_file", type=str, help="Archivo FASTA de la secuencia 1")
    parser.add_argument("seq2_file", type=str, help="Archivo FASTA de la secuencia 2")
    args = parser.parse_args()

    # Ejecutar el dotplot con los argumentos proporcionados
    run_dotplot(args.seq1_file, args.seq2_file)

Writing dotplot_mpi.py


In [11]:
!mpiexec -n 6 python3 dotplot_mpi.py E_coli.fna Salmonella.fna

Traceback (most recent call last):
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 58, in <module>
    run_dotplot(args.seq1_file, args.seq2_file)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 41, in run_dotplot
    dotplot = distributed_dotplot(seq1, seq2)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 15, in distributed_dotplot
    dotplot = np.zeros((n, m))
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 164. TiB for an array with shape (4641652, 4857450) and data type float64
Traceback (most recent call last):
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 58, in <module>
    run_dotplot(args.seq1_file, args.seq2_file)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 41, in run_dotplot
    dotplot = distributed_dotplot(seq1, seq2)
  File "/home/mrtopo/Doc